In [1]:
import math
import random
import string

In [2]:
random.seed(0)

# hitung bilangan acak dimana: a <= rand <b
def rand(a, b):
    return (b-a)*random.random() + a

In [3]:
# Buat matriks
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

In [4]:
# fungsi sigmoid
def sigmoid(x):
    return math.tanh(x)

In [5]:
# turunan dari fungsi sigmoid
def dsigmoid(y):
    return 1.0 - y**2

In [14]:
class NN:
    def __init__(self, ni, nh, no):
        # jumlah node masukan, tersembunyi, dan keluaran
        self.ni = ni + 1 # +1 untuk bias node
        self.nh = nh
        self.no = no

        # aktivasi nodes
        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no
        
        # bobot
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        
        # atur ke nilai acak
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2.0, 2.0)
        # perubahan terakhir bobot untuk momentum
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)
        
    def update(self, inputs):
        if len(inputs) != self.ni-1:
            raise ValueError('wrong number of inputs')

        # aktivasi input
        for i in range(self.ni-1):
            #self.ai[i] = sigmoid(inputs[i])
            self.ai[i] = inputs[i]

        # aktivasi hiden
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)

        # aktivasi output
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)

        return self.ao[:]
    
    def backPropagate(self, targets, N, M):
        if len(targets) != self.no:
            raise ValueError('wrong number of target values')

        # hitung istilah kesalahan untuk keluaran
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k]-self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # hitung istilah kesalahan untuk tersembunyi
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # perbarui bobot keluaran
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
                self.co[j][k] = change
                #print N*change, M*self.co[j][k]

        # perbarui bobot masukan
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
                self.ci[i][j] = change

        # hitung error
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k]-self.ao[k])**2
        return error
    
    def test(self, patterns):
        for p in patterns:
            print(p[0], '->', self.update(p[0]))
            
    def weights(self):
        print('Input weights:')
        for i in range(self.ni):
            print(self.wi[i])
        print()
        print('Output weights:')
        for j in range(self.nh):
            print(self.wo[j])
        
    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
            if i % 100 == 0:
                print('error %-.5f' % error)

In [16]:
def demo():
    # Ajarkan fungsi XOR jaringan
    pat = [
         [[0,0], [0]],
         [[0,1], [1]],
         [[1,0], [1]],
         [[1,1], [0]]
    ]
        
     # buat jaringan dengan dua node input, dua tersembunyi, dan satu output
    n = NN (2, 2, 1)
    # latih dengan beberapa pola
    n.train(pat)
     # test
    n.test(pat)
    
if __name__ == '__main__':
    demo()

error 0.70933
error 0.05400
error 0.00369
error 0.00169
error 0.00108
error 0.00079
error 0.00062
error 0.00811
error 0.00045
error 0.00039
[0, 0] -> [0.0017159179772160962]
[0, 1] -> [0.9818835310440086]
[1, 0] -> [0.9817423156444558]
[1, 1] -> [-0.0019188988666122486]
